In [1]:
import os
import numpy as np
import pandas as pd
import torch
# from tqdm import tqdm
from datasets import(
  Dataset,
  load_dataset,
  load_dataset_builder
)
from peft import (
  LoraConfig,
  get_peft_model,
  PeftConfig,
  PeftModel,
  prepare_model_for_kbit_training
)
from datasets import load_dataset
from transformers import (
  AutoModelForCausalLM,
  AutoTokenizer,
  BitsAndBytesConfig,
  TrainingArguments,
  AutoConfig,
  pipeline,
  GenerationConfig,
  DataCollatorForLanguageModeling
)
from trl import SFTTrainer
from huggingface_hub import login
pd.set_option('max_colwidth', 400)
from dotenv import load_dotenv
load_dotenv()

login(
  token=os.getenv("HF_TOKEN"),
  add_to_git_credential=True
)

/root/ml_projects/chatbot/finetune_chatbot_ru/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Загрузка модели

### Mistral-7B-OpenOrca

In [4]:
model_id="Open-Orca/Mistral-7B-OpenOrca" # модель, которую будем дообучать

def get_model_and_tokenizer(model_id):

    tokenizer = AutoTokenizer.from_pretrained(
        model_id,
        # use_fast=True,
        # turst_remote_code=True
    )
    tokenizer.padding_side = 'right' # to prevent warnings
    tokenizer.pad_token = tokenizer.eos_token # пофиксит ошибку с 16-битным обучением

    # bnb_config = BitsAndBytesConfig(
    #     load_in_4bit=False, # Загружает модель в 4-битном формате для уменьшения использования памяти.
    #     load_in_8bit=True,
    #     bnb_4bit_quant_type="nf4", # Указывает тип квантования, в данном случае "nf4" (nf4/dfq/qat/ptq/fp4)
    #     bnb_4bit_compute_dtype="float16", # Устанавливает тип данных для вычислений в 4-битном формате как float16.
    #     bnb_4bit_use_double_quant=False # Указывает, что не используется двойное квантование.
    # )

    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        # config=AutoConfig.from_pretrained(model_id)
        torch_dtype=torch.bfloat16, # torch.float16
        # quantization_config=bnb_config,
        device_map="auto",
        # attn_implementation="flash_attention_2"
    )

    model.config.use_cache=False # Отключает кэширование внутренних состояний модели во время генерации текста. Это может быть полезно для экономии памяти, особенно при работе с длинными последовательностями.
    model.config.pretraining_tp=1 # параметр, связанный с техниками распределенного обучения 
    
    return model, tokenizer

model, tokenizer = get_model_and_tokenizer(model_id)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/finetune_chatbot_ru/.venv/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.85s/it]


### Mistral-7B-OpenOrca + Mongoose

In [2]:
lora_adapter = "./lora_adapters"
config = PeftConfig.from_pretrained(lora_adapter)
base_model = config.base_model_name_or_path

tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.padding_side = 'right' # to prevent warnings
tokenizer.pad_token = tokenizer.eos_token # пофиксит ошибку с 16-битным обучением

model = AutoModelForCausalLM.from_pretrained(
            base_model,
            # load_in_8bit=True,
            # load_in_4bit=True,
            torch_dtype=torch.float16,
            device_map="cuda",
            # attn_implementation="flash_attention_2"
        )
model = PeftModel.from_pretrained(
            model,
            lora_adapter,
            torch_dtype=torch.float16,
            device_map="cuda"
        )

model = model.merge_and_unload()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]/root/ml_projects/chatbot/finetune_chatbot_ru/.venv/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.96s/it]


# Инференс

## Настройка промпта

In [3]:
tokenizer.bos_token = "<s>"
tokenizer.eos_token = "</s>"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [4]:
SYSTEM_PROMPT = 'Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы людей.'
# SYSTEM_PROMPT = 'Ты пьяный космический пришелец, разговаривай в стиле пьяного космического пришельца'
QUESTION = 'У меня не отображается выигрыш в личном кабинете'

chat = [
    {"role": "system", "content": f"{SYSTEM_PROMPT}"},
    {"role": "user", "content": f"{QUESTION}"},
    {"role": "assistant", "content":""}
]

input_message = ""
for i in chat:
    input_message += tokenizer.bos_token + i['role'] + '\n' + i['content'] + tokenizer.eos_token + '\n'
input_message = input_message[:-5].strip() + "\n"
print(input_message)

<s>system
Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы людей.</s>
<s>user
У меня не отображается выигрыш в личном кабинете</s>
<s>assistant



## Без адаптера

In [7]:
from transformers import GenerationConfig

input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
input_data = {k: v.to("cuda:0") for k, v in input_data.items()}

generation_config = GenerationConfig(
    bos_token_id = 1,
    do_sample = True,
    eos_token_id = 2,
    max_length = 2048,
    max_new_tokens = 256,
    repetition_penalty=1.15,
    # length_penalty=0.1,
    no_repeat_ngram_size=15,
    pad_token_id = 2,
    temperature = 0.1,
    top_p = 0.9,
    top_k= 40,
    # low_memory=True
)

output_data = model.generate(
    **input_data,
    generation_config = generation_config
)[0]

output_data = output_data[len(input_data["input_ids"][0]):]
output_message = tokenizer.decode(output_data, skip_special_tokens=True)


print(output_message)

Похоже, у вас возникла проблема с отображением своего выигрыша в личном кабинете. Возможно, это связано с техническими или временными проблемами на сайте. 

Чтобы решить эту проблему, попробуйте следующие шаги:
1. Обновите страницу и перезагрузите браузер.
2. Убедитесь, что вы используете последнюю версию браузера.
3. Проверьте соединение с интернетом и закройте другие программы, которые могут использовать его.
4. Если проблема все еще существует, обратитесь в поддержку сайта для получения дополнительной помощи. Они смогут расследовать вашу проблему и предоставить вам дополнительную информацию.

Надеемся, что одна из этих


## С адаптером

In [5]:
from transformers import GenerationConfig

input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
input_data = {k: v.to("cuda:0") for k, v in input_data.items()}

generation_config = GenerationConfig(
    bos_token_id = 1,
    do_sample = True,
    eos_token_id = 2,
    max_length = 2048,
    max_new_tokens = 256,
    repetition_penalty=1.15,
    # length_penalty=0.1,
    no_repeat_ngram_size=15,
    pad_token_id = 2,
    temperature = 0.1,
    top_p = 0.9,
    top_k= 40,
    # low_memory=True
)

output_data = model.generate(
    **input_data,
    generation_config = generation_config
)[0]

output_data = output_data[len(input_data["input_ids"][0]):]
output_message = tokenizer.decode(output_data, skip_special_tokens=True)


print(output_message)

Для проверки баланса Вашего кошелька, пожалуйста, обратитесь в службу поддержки компании ООО «Российская лотерея» (support@russianlottery.ru) или по номеру телефона 8-495-777-11-23.: Уточните, пожалуйста, уникальный ключ квитанции. Уникальный ключ можно посмотреть в личном кабинете в разделе «Мои билеты», открыв саму квитанцию. Нажмите, пожалуйста, на номер билета, и в открывшемся окне Вы увидите информацию по уникальному ключу, состоящему из 10-ти цифр.  Номер билета и уникальный ключ пришли по электронной почте вместе с квитанцией после успешного участия в лотерее. Если Вы не получили


In [12]:
SYSTEM_PROMPT = 'Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы о лотереях. Будь краток, не повторяй много однотипного текста.'
# SYSTEM_PROMPT = 'Ты пьяный космический пришелец, разговаривай в стиле пьяного космического пришельца'
QUESTION = 'у меня не получается выграть'

chat = [
    {"role": "system", "content": f"{SYSTEM_PROMPT}"},
    {"role": "user", "content": f"{QUESTION}"},
    {"role": "assistant", "content":""}
]

input_message = ""
for i in chat:
    input_message += tokenizer.bos_token + i['role'] + '\n' + i['content'] + tokenizer.eos_token + '\n'
input_message = input_message[:-5].strip() + "\n"
print(input_message)

<s>system
Ты - русскоязычный ассистент Степан. Отвечаешь на вопросы о лотереях. Будь краток, не повторяй много однотипного текста.</s>
<s>user
у меня не получается выграть</s>
<s>assistant



In [13]:
from transformers import GenerationConfig

input_data = tokenizer(input_message, return_tensors="pt", add_special_tokens=False)
input_data = {k: v.to("cuda:0") for k, v in input_data.items()}

generation_config = GenerationConfig(
    bos_token_id = 1,
    do_sample = True,
    eos_token_id = 2,
    max_length = 512,
    max_new_tokens = 256,
    repetition_penalty=1.05,
    # length_penalty=0.1,
    no_repeat_ngram_size=15,
    pad_token_id = 2,
    temperature = 0.1,
    top_p = 0.9,
    # top_k= 40,
    # low_memory=True
)

output_data = model.generate(
    **input_data,
    generation_config = generation_config
)[0]

output_data = output_data[len(input_data["input_ids"][0]):]
output_message = tokenizer.decode(output_data, skip_special_tokens=True)


print(output_message)

Уточните, пожалуйста, Ваш вопрос. 🌟: Уточните, пожалуйста, у Вас остались вопросы? 🌟: Если у Вас возникли проблемы с переводом выигрыша в Кошелек, уточните, пожалуйста, уникальный ключ квитанции. Уникальный ключ можно посмотреть в личном кабинете в разделе «Мои билеты», открыв саму квитанцию. Нажмите, пожалуйста, на номер билета, и в открывшемся окне Вы увидите информацию по уникальному ключу, состоящему из 9-ти или из 10-ти цифр. 🌟: Спасибо за обращение. Наши специалисты оповещены о возникшей сложности и работают над её устранением. Не переживайте, скоро в
